Dark Wala

In [10]:
from ctypes import *
import time
from sys import path
import sys
from os import sep
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

if sys.platform.startswith("win"):
    dwf = cdll.dwf
    constants_path = "C:" + sep + "Program Files (x86)" + sep + "Digilent" + sep + "WaveFormsSDK" + sep + "samples" + sep + "py"
elif sys.platform.startswith("darwin"):
    dwf = cdll.LoadLibrary("/Library/Frameworks/dwf.framework/dwf")
    constants_path = "/Applications/WaveForms.app/Contents/Resources/SDK/samples/py"
else:
    dwf = cdll.LoadLibrary("libdwf.so")
    constants_path = "/usr/share/digilent/waveforms/samples/py"

path.append(constants_path)
import dwfconstants as constants

In [11]:
dwf.FDwfDeviceCloseAll()
filter_flags = c_int(constants.enumfilterType.value | constants.enumfilterUSB.value)
device_count = c_int()
dwf.FDwfEnum(filter_flags, byref(device_count))

hdwf = c_int()
dwf.FDwfDeviceOpen(c_int(-1), byref(hdwf))
if hdwf.value == 0:
    raise RuntimeError("Failed to open device.")
print(hdwf.value)

dwf.FDwfAnalogIOReset(hdwf)
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(1), c_double(3.0))
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(0), c_double(1))
dwf.FDwfAnalogIOEnableSet(hdwf, c_int(1))
time.sleep(0.5)

vpos = c_double()
dwf.FDwfAnalogIOStatus(hdwf)
dwf.FDwfAnalogIOChannelNodeStatus(hdwf, c_int(0), c_int(1), byref(vpos))
print(f"Power: +{vpos.value:.2f}V")

1
Power: +3.00V


In [12]:
DEVICE_ADDR = 0x52
ENABLE_REG = 0x80
ATIME_REG = 0x81
CONTROL_REG = 0x8F
CDATAL_REG = 0x94
CDATAH_REG = 0x95
RDATAL_REG = 0x96
RDATAH_REG = 0x97
GDATAL_REG = 0x98
GDATAH_REG = 0x99
BDATAL_REG = 0x9A
BDATAH_REG = 0x9B

def write_register(reg, value):
    iNak = c_int()
    rgTX = (c_ubyte * 2)(reg, value)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(2), byref(iNak))
    return iNak.value == 0

def read_register(reg):
    iNak = c_int()
    rgTX = (c_ubyte * 1)(reg)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(1), byref(iNak))
    if iNak.value == 0:
        rgRX = (c_ubyte * 1)()
        dwf.FDwfDigitalI2cRead(hdwf, c_int(DEVICE_ADDR), rgRX, c_int(1), byref(iNak))
        if iNak.value == 0:
            return rgRX[0]
    return None

dwf.FDwfDigitalI2cRateSet(hdwf, c_double(100e3))
dwf.FDwfDigitalI2cSclSet(hdwf, c_int(0))
dwf.FDwfDigitalI2cSdaSet(hdwf, c_int(1))
iNak = c_int()
dwf.FDwfDigitalI2cClear(hdwf, byref(iNak))

1

In [13]:
write_register(ENABLE_REG, 0x01)
time.sleep(0.01)
write_register(ENABLE_REG, 0x03)
write_register(ATIME_REG, 0xFF)
write_register(CONTROL_REG, 0x02)

def read_colors():
    time.sleep(0.1)
    
    clear_low = read_register(CDATAL_REG)
    clear_high = read_register(CDATAH_REG)
    red_low = read_register(RDATAL_REG)
    red_high = read_register(RDATAH_REG)
    green_low = read_register(GDATAL_REG)
    green_high = read_register(GDATAH_REG)
    blue_low = read_register(BDATAL_REG)
    blue_high = read_register(BDATAH_REG)
    
    if all(val is not None for val in [clear_low, clear_high, red_low, red_high, green_low, green_high, blue_low, blue_high]):
        clear = (clear_high << 8) | clear_low
        red = (red_high << 8) | red_low
        green = (green_high << 8) | green_low
        blue = (blue_high << 8) | blue_low
        return {'clear': clear, 'red': red, 'green': green, 'blue': blue}
    return None

In [7]:
data1={'int_time': [], 'clear': [], 'red': [], 'green': [], 'blue': []}

write_register(ATIME_REG, 0xFF) # Integration Time value FF
atime_arr = [0xFF, 0xF6, 0xD5, 0xC0, 0x00]

for atime_values in atime_arr:
    write_register(ATIME_REG, atime_values)
    samples = []
    for i in range(100):
        Int_time = (256 - atime_values) * 2.4
        wait_time = (Int_time/1000)+ 0.01
        time.sleep(wait_time)
        
        colors = read_colors()
        if colors: samples.append(colors['clear'])
        data1['int_time'].append(Int_time)
        data1['clear'].append(colors['clear'])
        data1['red'].append(colors['red'])
        data1['green'].append(colors['green'])
        data1['blue'].append(colors['blue'])
        if i%10==0:
            print(f"Integration Time: {Int_time:.2f}: {colors['clear']}, R: {colors['red']}, G: {colors['green']}, B: {colors['blue']}")
        
    samples = np.array(samples)
    samples_mean = np.mean(samples)
    samples_std = np.std(samples)
    samples_jitter = samples_std / samples_mean * 100
    print(f"Mean: {samples_mean:.2f}, Std Dev: {samples_std:.2f}, Jitter: {samples_jitter:.2f}%")    
    print("-----"*10)

Integration Time: 2.40: 8, R: 5, G: 3, B: 2
Integration Time: 2.40: 8, R: 5, G: 3, B: 2
Integration Time: 2.40: 8, R: 5, G: 3, B: 2
Integration Time: 2.40: 8, R: 5, G: 3, B: 2
Integration Time: 2.40: 8, R: 5, G: 3, B: 2
Integration Time: 2.40: 8, R: 5, G: 3, B: 2
Integration Time: 2.40: 8, R: 5, G: 3, B: 2
Integration Time: 2.40: 8, R: 5, G: 3, B: 2
Integration Time: 2.40: 8, R: 5, G: 3, B: 2
Integration Time: 2.40: 8, R: 5, G: 3, B: 2
Mean: 8.00, Std Dev: 0.00, Jitter: 0.00%
--------------------------------------------------
Integration Time: 24.00: 86, R: 48, G: 27, B: 18
Integration Time: 24.00: 86, R: 48, G: 27, B: 18
Integration Time: 24.00: 86, R: 48, G: 27, B: 18
Integration Time: 24.00: 86, R: 48, G: 27, B: 18
Integration Time: 24.00: 86, R: 48, G: 27, B: 18
Integration Time: 24.00: 86, R: 48, G: 27, B: 18
Integration Time: 24.00: 86, R: 48, G: 27, B: 18
Integration Time: 24.00: 86, R: 48, G: 27, B: 18
Integration Time: 24.00: 86, R: 48, G: 27, B: 18
Integration Time: 24.00: 86

In [14]:
data1={'int_time': [], 'clear': [], 'red': [], 'green': [], 'blue': []}

write_register(ATIME_REG, 0xFF) # Integration Time value FF
gain_arr = {"x1": 0x00, "x4": 0x01, "x16": 0x02, "x64": 0x03}

for mult, gain_values in gain_arr.items():
    write_register(CONTROL_REG, gain_values)
    samples = []
    for i in range(100):
        Int_time = (256 - 0xFF) * 2.4
        wait_time = (Int_time/1000)+ 0.01
        time.sleep(wait_time)
        
        colors = read_colors()
        if colors: samples.append(colors['clear'])
        data1['int_time'].append(Int_time)
        data1['clear'].append(colors['clear'])
        data1['red'].append(colors['red'])
        data1['green'].append(colors['green'])
        data1['blue'].append(colors['blue'])
        if i%10==0:
            print(f"Gain: {mult}: {colors['clear']}, R: {colors['red']}, G: {colors['green']}, B: {colors['blue']}")
        
    samples = np.array(samples)
    samples_mean = np.mean(samples)
    samples_std = np.std(samples)
    samples_jitter = samples_std / samples_mean * 100
    print(f"Mean: {samples_mean:.2f}, Std Dev: {samples_std:.2f}, Jitter: {samples_jitter:.2f}%")    
    print("-----"*10)

Gain: x1: 0, R: 0, G: 0, B: 0
Gain: x1: 0, R: 0, G: 0, B: 0
Gain: x1: 0, R: 0, G: 0, B: 0
Gain: x1: 0, R: 0, G: 0, B: 0
Gain: x1: 0, R: 0, G: 0, B: 0
Gain: x1: 0, R: 0, G: 0, B: 0
Gain: x1: 0, R: 0, G: 0, B: 0
Gain: x1: 0, R: 0, G: 0, B: 0
Gain: x1: 0, R: 0, G: 0, B: 0
Gain: x1: 0, R: 0, G: 0, B: 0


C:\Users\ajayk\AppData\Local\Temp\ipykernel_24620\542568024.py:27: RuntimeWarning: invalid value encountered in scalar divide
  samples_jitter = samples_std / samples_mean * 100


Mean: 0.00, Std Dev: 0.00, Jitter: nan%
--------------------------------------------------
Gain: x4: 2, R: 1, G: 1, B: 1
Gain: x4: 2, R: 1, G: 1, B: 1
Gain: x4: 2, R: 1, G: 1, B: 1
Gain: x4: 2, R: 1, G: 1, B: 1
Gain: x4: 2, R: 1, G: 1, B: 1
Gain: x4: 2, R: 1, G: 1, B: 0
Gain: x4: 2, R: 1, G: 1, B: 1
Gain: x4: 2, R: 1, G: 1, B: 1
Gain: x4: 2, R: 1, G: 1, B: 1
Gain: x4: 2, R: 1, G: 1, B: 1
Mean: 2.00, Std Dev: 0.00, Jitter: 0.00%
--------------------------------------------------
Gain: x16: 8, R: 5, G: 3, B: 2
Gain: x16: 8, R: 5, G: 3, B: 2
Gain: x16: 8, R: 5, G: 3, B: 2
Gain: x16: 8, R: 5, G: 3, B: 2
Gain: x16: 8, R: 5, G: 3, B: 2
Gain: x16: 8, R: 5, G: 3, B: 2
Gain: x16: 8, R: 5, G: 3, B: 2
Gain: x16: 8, R: 5, G: 3, B: 2
Gain: x16: 9, R: 5, G: 3, B: 2
Gain: x16: 9, R: 5, G: 3, B: 2
Mean: 8.28, Std Dev: 0.45, Jitter: 5.42%
--------------------------------------------------
Gain: x64: 32, R: 18, G: 10, B: 7
Gain: x64: 33, R: 18, G: 10, B: 7
Gain: x64: 32, R: 18, G: 10, B: 7
Gain: x64: 33

In [15]:
def cleanup():
    dwf.FDwfDeviceClose(hdwf)
cleanup()